In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
#from sklearn.metrics import 

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

-----
### Train, valid split

In [17]:
pars_smp_train = np.load('data/pars_smp_train.npy')
y_smp_train = np.load('data/y_smp_train.npy')

In [18]:
pars_smp_train.shape, y_smp_train.shape

((1000000, 15, 1), (1000000, 200, 3))

In [23]:
y_smp_train[:100].shape

(100, 200, 3)

In [37]:
small_pars_smp_train = pars_smp_train[:100].copy()
small_y_smp_train = y_smp_train[:100].copy()

In [38]:
X_train, X_valid, y_train, y_valid = train_test_split(small_pars_smp_train, small_y_smp_train, test_size=0.2, shuffle=False, random_state=178)

In [39]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((80, 15, 1), (80, 200, 3), (20, 15, 1), (20, 200, 3))

In [40]:
class MacroEconomicModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(MacroEconomicModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Используйте только последний временной шаг для предсказания
        return out

# Определение размеров входных и выходных данных
input_size = 3  # Размерность данных о приросте ВВП, инфляции и процентной ставке
hidden_size = 64  # Размер скрытого состояния RNN
output_size = 15  # Размер параметров модели
num_layers = 2  # Количество слоев LSTM

# Создание экземпляра модели
model = MacroEconomicModel(input_size, hidden_size, output_size, num_layers)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [46]:
# Преобразование данных в тензоры PyTorch
y_train = torch.Tensor(y_train)
X_train = torch.Tensor(X_train)

y_valid = torch.Tensor(y_valid)
X_valid = torch.Tensor(X_valid)

batch_size = 15
train_dataset = TensorDataset(y_train, X_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

valid_dataset = TensorDataset(y_valid, X_valid)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Обучение модели
num_epochs = 10  # Количество эпох обучения
for epoch in range(num_epochs):
    batch_losses = []
    for batch in train_loader:
        inputs, targets = batch
        # Обнуляем градиенты
        optimizer.zero_grad()

        # Проход вперед (forward pass)
        outputs = model(inputs)

        # Вычисление функции потерь
        loss = torch.sqrt(criterion(outputs, targets.squeeze()))
        batch_losses.append(loss)

        loss.backward()
        optimizer.step()

    #training_loss = np.mean(batch_losses)
    
    with torch.no_grad():
        batch_val_losses = []
        for batch_val in valid_loader:
            inputs_val, targets_val = batch_val

            model.eval()

            # Проход вперед (forward pass)
            outputs_val = model(inputs)

            # Вычисление функции потерь
            loss_val = torch.sqrt(criterion(outputs_val, targets_val.squeeze()))
            batch_val_losses.append(loss_val)

            #validation_loss = np.mean(batch_val_losses)



    # Выводим информацию о процессе обучения
    #print(f'Эпоха [{epoch + 1}/{num_epochs}], Потери train: {loss.item()}')
    print(f'Эпоха [{epoch + 1}/{num_epochs}], Потери train: {batch_losses.item()}, Loss valid: {batch_val_losses.item()}')